# Smart signatures – Safety conditions
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

#### Check Purestake API

In [ ]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

## Clearing out modesty v3
* A donation to **Bob** that limits the amount that can be withdrawn in one transaction

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [ ]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

modesty_condition = And (
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() <= max_amount                       # Requested amount must be smaller than max_amount
)

fee_condition =  (Txn.fee() <= Int(1000))            # Max fee is 1000micro Algos (0.001 Algos)

saftey_condition = And (
        Txn.type_enum() == TxnType.Payment,          # Must be a "payment" transaction
        Global.group_size() == Int(1),               # Only 1 recipient
        Txn.rekey_to() == Global.zero_address()      # Cannot change private key
)

modest_pyteal = And(modesty_condition, fee_condition, saftey_condition)

##### Step 2: Compile PyTeal -> Teal

In [ ]:
modest_teal = compileTeal(modest_pyteal, Mode.Signature, version=3)
print(modest_teal)

##### Step 3: Compile Teal -> Bytecode for AVM

In [ ]:
Modest = algod_client.compile(modest_teal)
Modest

##### Step 4: Alice funds and deploys the smart signature

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modest['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

##### Step 5: Alice informs Bob

In [ ]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modest['result'])
print("Address of smart signature: ", Modest['hash'])

In [ ]:
# Check on Algoexplorer
print('https://testnet.algoexplorer.io/address/'+ Modest['hash'])

#### Step 6: Bob proposes a transaction that would withdraw too much

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- too much!!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)               # print entire error message

# Step 4: wait for confirmation
# No need to wait for confirmation
#txinfo = wait_for_confirmation(algod_client, txid)

#### Step 6: Bob withdraws smaller amount

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e6)              # <---------- within limit!!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 6b: Rekey attack

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0*1e6)              # <---------- zero!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt,
                 rekey_to = Bob['public'])

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 6c: TX fee attack

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
sp.fee = int(2.38e6)                         # <---------- WOW! 2 ALGO transaction fee
sp.flat_fee = True

withdrawal_amt = int(0.00001*1e6)             # <---------- small
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)